In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

## Purpose

The purpose of this notebook is to process the load data published as part of the Australian Government's *Smart City, Smart Grid* programme run in the city of Newcastle in NSW. This dataset is quite large and so the load profiles will be processed into an average daily profile using a distributed Dask cluster.

The original data can be found [here](http://datagovau.s3-ap-southeast-2.amazonaws.com/CDINTERVALREADINGALLNOQUOTES.csv.7z)


In [ ]:
file_name = "CD_INTERVAL_READING_ALL_NO_QUOTES.csv"
data_path = Path(f"./in/{file_name}")
out_path = Path("./out/aggregated.csv")
df = pd.read_csv(
    data_path.resolve(),
    parse_dates=["READING_DATETIME"],
)

In [ ]:
def aggregate_by_hour(df):
    df["READING_DATETIME"] = pd.to_datetime(df["READING_DATETIME"])
    time = df["READING_DATETIME"].dt.time
    df["time"] = time
    by_time = df.groupby(df["time"]).mean()
    return by_time

res = df.groupby("CUSTOMER_ID").apply(lambda x: aggregate_by_hour(x)).compute()
res.to_csv(out_path.resolve())
